# Workshop: Genetic Programming

The objective of this workshop is to implement a genetic programming algorithm to evolve a program that can solve the Lunar Lander problem.

The Lunar Lander problem is a classic reinforcement learning problem where the objective is to land a spacecraft on the moon. The spacecraft has two engines that can be used to control the spacecraft. The engines can be turned on or off, and the thrust of each engine can be set to one of three levels: -1, 0, or 1. The spacecraft has a limited amount of fuel, and the goal is to land the spacecraft on the moon with as little fuel as possible.

To use the Lunar Lander problem in this workshop, we will use the [OpenAI Gym](https://gym.openai.com/) library. The OpenAI Gym library provides a collection of reinforcement learning problems, including the Lunar Lander problem. The OpenAI Gym library also provides a Python interface to the Lunar Lander problem, which we will use in this workshop.

> Multiple problem can resolve during your installation of OpenAI Gym (If you have any problem with the Box2D installation, here are some [PythonH error](https://stackoverflow.com/questions/21530577/fatal-error-python-h-no-such-file-or-directory), [Swig error](https://stackoverflow.com/questions/54252800/python-cant-install-box2d-swig-exe-failed-with-error-code-1)

In [1]:
import gymnasium as gym

def get_env():
    return gym.make('LunarLander-v2', render_mode="rgb_array")


### Difference between Genetic Algorithm and Genetic Programming

To help you on how to implement the genetic programming algorithm, here are some differences between genetic algorithm and genetic programming:
1. Genetic programming uses a tree structure to represent the solution, while genetic algorithm uses a vector of values. The numbers of values can be different for each solution in Genetic Programming, while the number of values is the same for each solution in Genetic Algorithm.
2. The mutation operator is different. In genetic programming, the mutation operator can change the structure of the tree, while in genetic algorithm, the mutation operator can only change the values in the vector.


#### Exercise 1: Create an individual class

The first step is to create a class to represent an individual in the population. The individual class will represent a program that can solve the Lunar Lander problem. The individual class will also be used to represent the population of individuals.

The individual class will have the following attributes:
1. `env`: Its own environment to evaluate the fitness.
2. `Instructions list`: The list of instructions that make up the program with instructions at the creation.
3. `Fitness`: The fitness of the individual (setted while the evaluation).
4. `Evaluate method`: The method that will evaluate the individual.
5. `Mutate method`: The method that will mutate the individual.

> If you evaluate the individual with a setted seed, you will have better training results (because the environment will be the same for each individual).

In [2]:
class Individual():
    def __init__(self) -> None:
        self.env = get_env()
        self.instructions = [self.env.action_space.sample() for _ in range(200)]
        self.fitness = 0
        self.depth = 200

    def evaluate(self):
        self.env.reset(seed=42)
        for instruction in self.instructions:
            observation, reward, done, truncated, info = self.env.step(instruction)
            self.fitness += reward
            if done:
                break
        return self.fitness


#### Exercise 2: Create a population class

The second step is to create a class to represent a population of individuals. The population class will represent a population of programs that can solve the Lunar Lander problem. This will be the main class of the genetic programming algorithm.

First, the population class will have the following attributes:
1. `a population of individuals`: The list of individuals that make up the population. (setted at the creation)
2. `a population size`: The size of the population. (setted at the creation)
3. `a best individual`: The best individual in the population. (setted during the evaluation)
4. `an evaluation method`: The method that will evaluate the population.
5. `a selection method`: The method that will select the individuals to reproduce.
6. `a reproduction method`: The method that will reproduce the individuals.
7. `a mutation method`: The method that will mutate the individuals.

In [3]:
import random

class Population():
    def __init__(self, size) -> None:
        self.size = size
        self.individuals = [Individual() for _ in range(size)]
        self.generation = 0
        self.best = None
        self.best_fitness = -1000

    def evaluate(self):
        for individual in self.individuals:
            individual.evaluate()
            if individual.fitness > self.best_fitness:
                self.best = individual
                self.best_fitness = individual.fitness
    
    def select(self):
        self.individuals.sort(key=lambda x: x.fitness, reverse=True)
        self.individuals = self.individuals[:self.size//2]
        self.size = len(self.individuals)
    
    def crossover(self):
        for i in range(self.size):
            parent1 = self.individuals[0]
            parent2 = self.individuals[i]
            child = Individual()
            random_index = random.randint(0, parent1.depth-1)
            child.instructions = parent1.instructions[:random_index] + parent2.instructions[random_index:]
            child.depth = len(child.instructions)
            self.individuals.append(child)
        self.size = len(self.individuals)
    
    def mutate(self):
        for individual in self.individuals:
            random_index = random.randint(0, individual.depth-1)
            if (random.random() < 0.1):
                individual.instructions[random_index] = individual.env.action_space.sample()
            elif (random.random() < 0.4):
                individual.instructions.append(individual.env.action_space.sample())
                individual.depth += 1
            elif (random.random() < 0.45 and individual.depth > 1):
                individual.instructions.pop(random_index)
                individual.depth -= 1

### Exercise 3: Create a main function

The third step is to create a main function to run the genetic programming algorithm. The main function will create a population of individuals, evaluate the population, and evolve the population.

In [19]:
population = Population(40)
while population.generation < 1:
    population.evaluate()
    print(f"Generation: {population.generation} - Best Fitness: {population.best_fitness} - Number of Instructions: {population.best.depth}")
    population.select()
    population.crossover()
    population.mutate()
    population.generation += 1
print(population.best.instructions)

Generation: 0 - Best Fitness: -74.6592100523479 - Number of Instructions: 200
[2, 1, 3, 1, 2, 1, 1, 0, 2, 1, 3, 3, 2, 0, 2, 2, 1, 2, 2, 1, 0, 2, 0, 0, 2, 1, 0, 0, 3, 2, 3, 0, 0, 1, 0, 0, 1, 0, 3, 3, 0, 3, 0, 3, 1, 1, 1, 2, 2, 1, 0, 1, 1, 0, 3, 3, 1, 1, 2, 2, 2, 0, 3, 3, 0, 2, 3, 2, 1, 2, 2, 0, 3, 0, 1, 3, 3, 3, 0, 1, 2, 0, 3, 0, 2, 2, 3, 2, 3, 2, 2, 0, 2, 0, 2, 0, 0, 2, 2, 2, 0, 2, 2, 2, 0, 3, 0, 3, 0, 3, 3, 1, 0, 3, 0, 3, 1, 2, 0, 1, 0, 2, 3, 0, 1, 0, 0, 2, 0, 3, 0, 2, 3, 3, 1, 2, 3, 2, 2, 3, 2, 2, 1, 2, 0, 3, 0, 3, 3, 0, 2, 0, 1, 0, 3, 3, 3, 3, 1, 0, 3, 1, 3, 1, 1, 2, 2, 3, 3, 2, 2, 2, 3, 1, 3, 1, 2, 2, 2, 2, 1, 3, 0, 1, 2, 3, 0, 0, 1, 1, 2, 0, 0, 3, 2, 3, 3, 1, 1, 0]


### Exercise 4: Visualize the results

The fourth step is to visualize the results of the genetic programming algorithm. The visualization will show the best individual in the population at each generation.

In [18]:
population.best.env = gym.make('LunarLander-v2', render_mode="human")
population.best.evaluate()
population.best.env.close()

#### Exercise 5: Manipulate the parameters

Now that you have implemented the genetic programming algorithm, you can manipulate the parameters of the algorithm to see how they affect the results. You can manipulate the following parameters:
1. `Population size`: The size of the population.
2. `Number of generations`: The number of generations to evolve the population.
3. `Mutation rate`: The probability that an individual will mutate.
4. `Number of instructions`: The number of instructions in the program.
5. `Crossover type`: The type of crossover to use to reproduce the individuals.
6. `Selection type`: The type of selection to use to select the individuals to reproduce.
7. `Mutation type`: The type of mutation to use to mutate the individuals.

You can consider that you have a good result when the fitness of the best individual is greater thant 3000. But your goal is to have the best fitness possible.

> A lot of parameters can be used to have the best results. Try to accelerate the convergence of the algorithm by manipulating the parameters.

# Conclusion

In this workshop, you have implemented a simple genetic programming algorithm to solve the Lunar Lander problem. You have also learned how to manipulate the parameters of the algorithm to see how they affect the results.

# To go further

To go further, you can try to implement a more complex genetic programming algorithm to solve the Lunar Lander problem. You can also try to implement a genetic programming algorithm to solve another reinforcement learning problem.

As example, you can try to implement it to manipulate neural networks. You can also try to implement it to solve the CartPole problem.